## Building and Serving an OpenAI compatiable Chatbot

In [1]:
!pip install mlflow>=2.11.0 -q -U
# OpenAI-compatible chat model support is available for Transformers 4.34.0 and above
!pip install transformers>=4.34.0 -q -U

  You can safely remove it manually.


In [2]:
# Disable tokenizers warnings when constructing pipelines
%env TOKENIZERS_PARALLELISM=false

import warnings

# Disable a few less-than-useful UserWarnings from setuptools and pydantic
warnings.filterwarnings("ignore", category = UserWarning)

env: TOKENIZERS_PARALLELISM=false


## Building a Chat Model

In [ ]:
from transformers import pipeline

import mlflow

generator = pipeline(
    "text-generation",
    model = "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
)

# save the model using the vanilla `text-generation` task type
mlflow.transformers.save_model(
    path = "tinyllama-text-generation", transformers_model = generator, task = "text-generation"
)

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.20k [00:00<?, ?B/s]

2024/11/18 16:19:55 WARNING mlflow.utils.environment: On Windows, timeout is not supported for model requirement inference. Therefore, the operation is not bound by a timeout and may hang indefinitely. If it hangs, please consider specifying the signature manually.


In [5]:
# Load the model for inference 
model = mlflow.pyfunc.load_model('tinyllama-text-generation')

model.metadata.signature

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

inputs: 
  [string (required)]
outputs: 
  [string (required)]
params: 
  None

In [7]:
# first, apply the tokenizer's chat template, since the
# model is tuned to accept prompts in a chat format. this
# also converts the list of messages to a string.
messages = [{"role": "user", "content": "Write me a hello world program in python"}]
prompt = generator.tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

model.predict(prompt)

['<|user|>\nWrite me a hello world program in python</s>\n<|assistant|>\nHere\'s a simple hello world program in Python:\n\n```python\nprint("Hello, world!")\n```\n\nThis program prints the string "Hello, world!" to the console.\n\nTo run this program, simply run it from the command line or a text editor. For example:\n\n```\npython\n>>> print("Hello, world!")\nHello, world!\n```']

In [8]:
mlflow.transformers.save_model(
    path = "tinyllama-chat", transformers_model = generator, task = "llm/v1/chat"
)

2024/11/18 17:21:18 WARNING mlflow.utils.environment: On Windows, timeout is not supported for model requirement inference. Therefore, the operation is not bound by a timeout and may hang indefinitely. If it hangs, please consider specifying the signature manually.


In [9]:
model = mlflow.pyfunc.load_model("tinyllama-chat")
model.metadata.signature

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

inputs: 
  ['messages': Array({content: string (optional), name: string (optional), refusal: string (optional), role: string (required), tool_call_id: string (optional), tool_calls: Array({function: {arguments: string (required), name: string (required)} (required), id: string (required), type: string (required)}) (optional)}) (required), 'temperature': double (optional), 'max_tokens': long (optional), 'stop': Array(string) (optional), 'n': long (optional), 'stream': boolean (optional), 'top_p': double (optional), 'top_k': long (optional), 'frequency_penalty': double (optional), 'presence_penalty': double (optional), 'tools': Array({function: {description: string (optional), name: string (required), parameters: {additionalProperties: boolean (optional), properties: Map(str -> {description: string (optional), enum: Array(string) (optional), items: {type: string (required)} (optional), type: string (required)}) (required), required: Array(string) (optional), type: string (optional)} (req

In [10]:
messages = [{'role': 'user', "content": "Write me a hello world program in python"}]

model.predict({"messages": messages})

[{'id': '2b3a58fa-5dc9-4c30-ab82-6fd2d43f1c89',
  'object': 'chat.completion',
  'created': 1731930788,
  'model': 'TinyLlama/TinyLlama-1.1B-Chat-v1.0',
  'usage': {'prompt_tokens': 24, 'completion_tokens': 87, 'total_tokens': 111},
  'choices': [{'index': 0,
    'finish_reason': 'stop',
    'message': {'role': 'assistant',
     'content': 'Here\'s a simple hello world program in Python:\n\n```python\nprint("Hello, world!")\n```\n\nThis program prints the string "Hello, world!" to the console.\n\nTo run this program, simply run it from the command line or a text editor. For example:\n\n```\npython\n>>> print("Hello, world!")\nHello, world!\n```'}}]}]